# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader
import os

In [2]:
expt_dict = {}
for n in range(10,14):    
    expt_dict[f'ND00{n}']=[f'Pos{i}' for i in range(14)]

In [ ]:
for expt in ['ND0010']:#expt_dict:
    for pos in expt_dict[expt]:
        image_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/{pos}_images'
        transform_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor_1.npy'
        if os.path.exists(image_path):
            print(expt, pos)
            images = DaskOctopusLiteLoader(image_path, remove_background=False, 
                                           transforms=transform_path,
                                           crop = (1200, 1600))
            gfp = images['gfp']
            rfp = images['rfp']
            irfp = images['irfp']
            masks = images['mask']
            viewer = napari.Viewer()
            viewer.add_image(gfp, name="gfp", 
                             contrast_limits=[0,255],
                             blending = 'additive', colormap = 'green')#, is_pyramid=False)
            viewer.add_image(rfp, name="rfp", 
                             contrast_limits=[0,55], 
                             blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
            viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
            viewer.add_image(masks, name = f'trans {expt}/{pos}',
                            contrast_limits=[0,2],
                           blending = 'additive',)
            viewer.show(block=True)
        transform_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor.npy'
        if os.path.exists(image_path):
            print(expt, pos)
            images = DaskOctopusLiteLoader(image_path, remove_background=False, 
                                           transforms=transform_path,
                                           crop = (1200, 1600))
            gfp = images['gfp']
            rfp = images['rfp']
            irfp = images['irfp']
            masks = images['mask']
            viewer = napari.Viewer()
            viewer.add_image(gfp, name="gfp", 
                             contrast_limits=[0,255],
                             blending = 'additive', colormap = 'green')#, is_pyramid=False)
            viewer.add_image(rfp, name="rfp", 
                             contrast_limits=[0,55], 
                             blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
            viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
            viewer.add_image(masks, name = f'/{expt}/{pos}',
                            contrast_limits=[0,2],
                           blending = 'additive',)
            viewer.show(block=True)

ND0010 Pos0
Using cropping: (1200, 1600)
ND0010 Pos0
Using cropping: (1200, 1600)


Exception in callback BaseAsyncIOLoop._handle_events(84, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(84, 1)>
Traceback (most recent call last):
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/nathan/ana

ND0010 Pos1
Using cropping: (1200, 1600)


# Load images

In [2]:
# define experiment ID and select a position
expt = 'ND0011'
pos = 'Pos6'
# point to where the data is
root_dir = '/home/nathan/data'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'
# lazily load images
images = DaskOctopusLiteLoader(image_path, 
                               transforms = transform_path,
                               crop = (1200,1600),
                               remove_background = False)
images.channels

Using cropping: (1200, 1600)


[<Channels.BRIGHTFIELD: 0>,
 <Channels.GFP: 1>,
 <Channels.RFP: 2>,
 <Channels.IRFP: 3>,
 <Channels.MASK: 99>]

In [9]:
gfp = images['gfp']
rfp = images['rfp']
irfp = images['irfp']
masks = images['mask']

### Add tracks

In [4]:
import btrack
from btrack.utils import tracks_to_napari

#### iRFP tracks

In [7]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    irfp_tracks = h.tracks

[INFO][2022/01/13 06:25:36 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0009/Pos3/gfp_tracks.h5...
[INFO][2022/01/13 06:25:36 PM] Loading tracks/obj_type_1
[INFO][2022/01/13 06:25:36 PM] Loading objects/obj_type_1 (195987, 5) (195987 filtered: None)
[INFO][2022/01/13 06:25:41 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0009/Pos3/gfp_tracks.h5


In [8]:
irfp_tracks, irfp_properties, irfp_graph = tracks_to_napari(irfp_tracks, ndim = 2)

#### RFP tracks

In [7]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
    rfp_tracks = h.tracks

[INFO][2022/01/13 06:25:36 PM] Opening HDF file: /home/nathan/data/kraken/ras/ND0009/Pos3/gfp_tracks.h5...
[INFO][2022/01/13 06:25:36 PM] Loading tracks/obj_type_1
[INFO][2022/01/13 06:25:36 PM] Loading objects/obj_type_1 (195987, 5) (195987 filtered: None)
[INFO][2022/01/13 06:25:41 PM] Closing HDF file: /home/nathan/data/kraken/ras/ND0009/Pos3/gfp_tracks.h5


In [8]:
rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(rfp_tracks, ndim = 2)

# Launch viewer

In [12]:
viewer = napari.Viewer()
viewer.add_image(gfp, name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp, name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
viewer.add_image(masks, name = 'masks',
                contrast_limits=[0,255],
               blending = 'additive',)
viewer.add_tracks(irfp_tracks)
viewer.add_tracks(rfp_tracks)

<Tracks layer 'gfp_tracks' at 0x7f2c4da63910>